## DIA 024: Implementación del Registro y Gestión de Usuarios con una Base de Datos

1. Objetivos del Día 24
Configurar una Base de Datos para Gestión de Usuarios:
Integrar una base de datos para almacenar información de usuarios de manera persistente.
Implementar el Registro de Usuarios:
Crear endpoints que permitan a los nuevos usuarios registrarse proporcionando sus credenciales.
Almacenar Contraseñas de Manera Segura:
Utilizar hashing para almacenar contraseñas de forma segura en la base de datos.
Actualizar el Endpoint de Login:
Modificar el proceso de autenticación para validar credenciales contra la base de datos.
Actualizar las Pruebas Automatizadas:
Añadir pruebas para asegurar que el registro y la autenticación de usuarios funcionan correctamente.
2. Configurar la Base de Datos con SQLAlchemy
2.1. Instalar Dependencias Necesarias
Para interactuar con una base de datos de manera eficiente, utilizaremos SQLAlchemy junto con Flask-Migrate para manejar las migraciones de la base de datos. Además, usaremos Flask-Bcrypt para el hashing seguro de contraseñas.

Añade las siguientes dependencias a tu archivo requirements.txt:

plaintext
Copiar
Flask-SQLAlchemy==3.0.3
Flask-Migrate==4.0.4
Flask-Bcrypt==1.0.1
Instalar las Dependencias:

bash
Copiar
pip install -r requirements.txt
2.2. Configurar SQLAlchemy en api.py
Actualiza tu archivo api.py para configurar SQLAlchemy, Flask-Migrate y Flask-Bcrypt.

python
Copiar
# api.py

from flask import Flask, request, jsonify, render_template
import tensorflow as tf
from PIL import Image
import numpy as np
import io
import os
import redis
import hashlib
from functools import wraps
from flask_jwt_extended import JWTManager, create_access_token, jwt_required, get_jwt_identity
from flask_limiter import Limiter
from flask_limiter.util import get_remote_address
from flask_sqlalchemy import SQLAlchemy
from flask_migrate import Migrate
from flask_bcrypt import Bcrypt

app = Flask(__name__)

# Configuración de JWT
app.config['JWT_SECRET_KEY'] = os.getenv('JWT_SECRET_KEY', 'super-secret-key')  # Cambia esto a una clave segura
jwt = JWTManager(app)

# Configuración de Rate Limiting
limiter = Limiter(
    app,
    key_func=get_remote_address,
    default_limits=["200 per day", "50 per hour"],
    default_limits_exempt=["/login", "/register"]  # Exento los endpoints de login y registro
)

# Configuración de la Base de Datos
app.config['SQLALCHEMY_DATABASE_URI'] = os.getenv('DATABASE_URL', 'sqlite:///users.db')  # Usa PostgreSQL en producción
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)
migrate = Migrate(app, db)

# Configuración de Bcrypt
bcrypt = Bcrypt(app)

# Configurar Redis
redis_url = os.getenv('REDIS_URL', 'redis://localhost:6379/0')
cache = redis.Redis.from_url(redis_url)

# Cargar el modelo optimizado
modelo_web = tf.keras.models.load_model('modelo_cnn_transfer_learning_mnist_final_optimizado.h5')

# Definir el modelo de Usuario
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    password = db.Column(db.String(128), nullable=False)

    def __repr__(self):
        return f'<User {self.username}>'

def allowed_file(filename):
    ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg', 'gif'}
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

def generate_cache_key(file_bytes):
    return hashlib.sha256(file_bytes).hexdigest()

@app.route('/')
def home():
    return render_template('index.html')
Explicación:

Configuración de la Base de Datos:
SQLALCHEMY_DATABASE_URI: Define la URI de la base de datos. En desarrollo, usamos SQLite; en producción, es recomendable usar PostgreSQL u otra base de datos robusta.
SQLALCHEMY_TRACK_MODIFICATIONS: Desactiva el seguimiento de modificaciones para mejorar el rendimiento.
Inicialización de Extensiones:
db: Inicializa SQLAlchemy para gestionar la base de datos.
migrate: Inicializa Flask-Migrate para manejar las migraciones de la base de datos.
bcrypt: Inicializa Flask-Bcrypt para el hashing seguro de contraseñas.
Modelo de Usuario:
Define una clase User que representa a los usuarios en la base de datos, con campos para id, username y password.
2.3. Crear el Modelo de Usuario
El modelo User ya se ha definido en el paso anterior. Asegúrate de que está incluido en tu archivo api.py:

python
Copiar
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    password = db.Column(db.String(128), nullable=False)

    def __repr__(self):
        return f'<User {self.username}>'
Explicación:

Campos del Modelo:
id: Identificador único para cada usuario.
username: Nombre de usuario único.
password: Contraseña almacenada de manera segura (hashed).
2.4. Inicializar la Base de Datos
Antes de crear los endpoints, debemos inicializar la base de datos y aplicar las migraciones.

Inicializar Migraciones:

bash
Copiar
flask db init
Crear una Migración:

bash
Copiar
flask db migrate -m "Crear modelo de Usuario"
Aplicar la Migración:

bash
Copiar
flask db upgrade
Explicación:

flask db init: Inicializa el directorio de migraciones.
flask db migrate: Genera una nueva migración basada en los cambios en los modelos.
flask db upgrade: Aplica la migración a la base de datos, creando las tablas necesarias.
3. Implementar el Registro de Usuarios
3.1. Crear el Endpoint de Registro
Añade un nuevo endpoint /register que permita a los usuarios registrarse proporcionando un nombre de usuario y una contraseña.

python
Copiar
@app.route('/register', methods=['POST'])
def register():
    """
    Endpoint para registrar nuevos usuarios.
    """
    data = request.get_json()
    username = data.get('username', None)
    password = data.get('password', None)
    
    if not username or not password:
        return jsonify({"msg": "Username and password required"}), 400
    
    # Verificar si el usuario ya existe
    if User.query.filter_by(username=username).first():
        return jsonify({"msg": "Username already exists"}), 409
    
    # Hash de la contraseña
    hashed_password = bcrypt.generate_password_hash(password).decode('utf-8')
    
    # Crear un nuevo usuario
    new_user = User(username=username, password=hashed_password)
    db.session.add(new_user)
    db.session.commit()
    
    return jsonify({"msg": "User registered successfully"}), 201
Explicación:

Validación de Datos:
Verifica que se hayan proporcionado tanto username como password.
Verificación de Usuario Existente:
Comprueba si el username ya está en uso para evitar duplicados.
Hash de Contraseña:
Utiliza bcrypt para generar un hash seguro de la contraseña antes de almacenarla.
Creación y Almacenamiento del Usuario:
Crea una instancia de User con el username y la contraseña hasheada.
Añade el nuevo usuario a la sesión de la base de datos y confirma los cambios.
3.2. Validar y Almacenar Datos de Usuario
El endpoint de registro ya maneja la validación de datos y el almacenamiento seguro de las contraseñas. Asegúrate de que las solicitudes al endpoint /register incluyan un cuerpo JSON con username y password.

Ejemplo de Solicitud de Registro:

bash
Copiar
curl -X POST http://localhost:5000/register \
     -H "Content-Type: application/json" \
     -d '{"username": "nuevo_usuario", "password": "contrasena_segura"}'
Respuesta Esperada:

json
Copiar
{
    "msg": "User registered successfully"
}
4. Actualizar el Endpoint de Login para Validar Credenciales desde la Base de Datos
Modifica el endpoint /login para autenticar a los usuarios verificando las credenciales contra la base de datos en lugar de usar una API Key estática.

python
Copiar
@app.route('/login', methods=['POST'])
def login():
    """
    Endpoint para que los usuarios inicien sesión y obtengan un token JWT.
    """
    data = request.get_json()
    username = data.get('username', None)
    password = data.get('password', None)
    
    if not username or not password:
        return jsonify({"msg": "Username and password required"}), 400
    
    # Buscar al usuario en la base de datos
    user = User.query.filter_by(username=username).first()
    
    if not user or not bcrypt.check_password_hash(user.password, password):
        return jsonify({"msg": "Bad username or password"}), 401
    
    # Crear el token de acceso
    access_token = create_access_token(identity=username)
    return jsonify(access_token=access_token), 200
Explicación:

Validación de Datos:
Verifica que se hayan proporcionado tanto username como password.
Autenticación de Usuario:
Busca al usuario en la base de datos usando el username.
Utiliza bcrypt para verificar que la contraseña proporcionada coincide con la almacenada.
Generación del Token JWT:
Si las credenciales son válidas, se genera un token JWT que el usuario puede usar para autenticar futuras solicitudes.
Respuesta:
Devuelve el token JWT en formato JSON si la autenticación es exitosa.
5. Actualizar las Pruebas Automatizadas para el Registro de Usuarios
Es fundamental asegurarse de que los nuevos endpoints de registro y autenticación funcionen correctamente. Actualizaremos las pruebas automatizadas para cubrir estos casos.

5.1. Actualizar test_api.py para Registro y Autenticación JWT
Actualiza el archivo tests/test_api.py para incluir pruebas de registro de usuarios y validación de autenticación.

python
Copiar
# tests/test_api.py

import pytest
from api import app, db, User
import os
from io import BytesIO
from PIL import Image
import numpy as np
import json

@pytest.fixture
def client():
    app.config['TESTING'] = True
    app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///:memory:'  # Usar una base de datos en memoria para pruebas
    with app.test_client() as client:
        with app.app_context():
            db.create_all()
            yield client
            db.session.remove()
            db.drop_all()

def generate_image(digit=5):
    # Genera una imagen en blanco con un dígito negro en el centro
    img = Image.new('L', (28, 28), color=255)
    # Aquí podrías dibujar el dígito, pero para simplicidad, dejaremos la imagen en blanco
    # En un caso real, usarías una imagen real de MNIST
    return img

def register_user(client, username, password):
    response = client.post('/register', json={
        'username': username,
        'password': password
    })
    return response

def login(client, username, password):
    response = client.post('/login', json={
        'username': username,
        'password': password
    })
    data = response.get_json()
    return data.get('access_token')

def test_register_success(client):
    response = register_user(client, 'testuser', 'testpassword')
    assert response.status_code == 201
    json_data = response.get_json()
    assert json_data['msg'] == 'User registered successfully'

def test_register_existing_user(client):
    # Registrar el primer usuario
    response = register_user(client, 'testuser', 'testpassword')
    assert response.status_code == 201
    
    # Intentar registrar el mismo usuario nuevamente
    response = register_user(client, 'testuser', 'nueva_contrasena')
    assert response.status_code == 409
    json_data = response.get_json()
    assert json_data['msg'] == 'Username already exists'

def test_login_success(client):
    # Registrar un usuario
    register_user(client, 'testuser', 'testpassword')
    
    # Iniciar sesión
    token = login(client, 'testuser', 'testpassword')
    assert token is not None

def test_login_bad_credentials(client):
    # Registrar un usuario
    register_user(client, 'testuser', 'testpassword')
    
    # Intentar iniciar sesión con contraseña incorrecta
    response = client.post('/login', json={
        'username': 'testuser',
        'password': 'wrongpassword'
    })
    assert response.status_code == 401
    json_data = response.get_json()
    assert json_data['msg'] == 'Bad username or password'

def test_predict_success(client):
    # Registrar y iniciar sesión
    register_user(client, 'testuser', 'testpassword')
    token = login(client, 'testuser', 'testpassword')
    assert token is not None
    
    img = generate_image()
    img_byte_arr = BytesIO()
    img.save(img_byte_arr, format='PNG')
    img_byte_arr.seek(0)
    
    data = {
        'file': (img_byte_arr, 'test.png')
    }
    headers = {
        'Authorization': f'Bearer {token}'
    }
    
    response = client.post('/predict', data=data, headers=headers, content_type='multipart/form-data')
    assert response.status_code == 200
    json_data = response.get_json()
    assert 'prediccion' in json_data
    assert 'probabilidad' in json_data
    assert 'cached' in json_data
    assert isinstance(json_data['prediccion'], int)
    assert isinstance(json_data['probabilidad'], float)

def test_predict_no_file(client):
    # Registrar y iniciar sesión
    register_user(client, 'testuser', 'testpassword')
    token = login(client, 'testuser', 'testpassword')
    assert token is not None
    
    data = {}
    headers = {
        'Authorization': f'Bearer {token}'
    }
    
    response = client.post('/predict', data=data, headers=headers, content_type='multipart/form-data')
    assert response.status_code == 400
    json_data = response.get_json()
    assert json_data['error'] == 'No se encontró el archivo en la solicitud.'

def test_predict_invalid_file_type(client):
    # Registrar y iniciar sesión
    register_user(client, 'testuser', 'testpassword')
    token = login(client, 'testuser', 'testpassword')
    assert token is not None
    
    # Crear un archivo de texto en lugar de una imagen
    file_data = BytesIO(b"Este no es una imagen válida.")
    data = {
        'file': (file_data, 'test.txt')
    }
    headers = {
        'Authorization': f'Bearer {token}'
    }
    
    response = client.post('/predict', data=data, headers=headers, content_type='multipart/form-data')
    assert response.status_code == 400
    json_data = response.get_json()
    assert json_data['error'] == 'Tipo de archivo no permitido. Por favor, sube una imagen PNG, JPG, JPEG o GIF.'

def test_predict_unauthorized(client):
    # Intentar acceder al endpoint sin token
    img = generate_image()
    img_byte_arr = BytesIO()
    img.save(img_byte_arr, format='PNG')
    img_byte_arr.seek(0)
    
    data = {
        'file': (img_byte_arr, 'test.png')
    }
    
    response = client.post('/predict', data=data, content_type='multipart/form-data')
    assert response.status_code == 401
    json_data = response.get_json()
    assert json_data['msg'] == 'Missing Authorization Header'
Explicación:

Configuración de la Base de Datos para Pruebas:
Usa una base de datos en memoria (sqlite:///:memory:) para aislar las pruebas y evitar afectar la base de datos de desarrollo o producción.
Funciones Auxiliares:
register_user: Facilita el registro de usuarios en las pruebas.
login: Facilita la obtención de tokens JWT para las pruebas.
Nuevas Pruebas Añadidas:
test_register_success: Verifica que un nuevo usuario pueda registrarse exitosamente.
test_register_existing_user: Asegura que intentar registrar un usuario con un username ya existente falle con un error apropiado.
test_login_success: Comprueba que un usuario registrado pueda iniciar sesión y obtener un token JWT.
test_login_bad_credentials: Verifica que iniciar sesión con credenciales incorrectas falle con un error adecuado.
Actualización de Pruebas Existentes:
test_predict_success: Ahora incluye el registro e inicio de sesión para obtener un token JWT antes de realizar la predicción.
6. Conclusiones y Recomendaciones
6.1. Implementación de Registro y Gestión de Usuarios
Beneficios:

Gestión Centralizada de Usuarios: Permite administrar usuarios de manera eficiente y escalable.
Seguridad Mejorada: Almacenar contraseñas de forma segura mediante hashing protege la información sensible.
Autenticación Robustecida: JWT proporciona una forma segura y escalable de autenticar a los usuarios sin necesidad de almacenar sesiones en el servidor.
Recomendaciones:

Validación Avanzada de Datos: Implementa validaciones más estrictas para los campos de registro, como longitud de contraseña, formatos de username, etc.
Gestión de Roles y Permisos: Considera añadir roles (e.g., admin, user) para controlar el acceso a diferentes funcionalidades de la API.
Implementar Confirmación de Email: Añade un mecanismo para verificar la dirección de correo electrónico de los usuarios durante el registro.
6.2. Almacenamiento Seguro de Contraseñas
Beneficios:
Protección contra Brechas de Seguridad: Almacenar contraseñas de manera hasheada reduce el riesgo en caso de una brecha de datos.
Recomendaciones:
Uso de Salting y Peppering: Además del hashing, considera agregar un salt único por usuario y un pepper global para incrementar la seguridad.
Actualización de Hashing: Mantente actualizado con las mejores prácticas y algoritmos de hashing para contraseñas.
6.3. Actualización de Pruebas Automatizadas
Beneficios:

Garantía de Funcionalidad Correcta: Asegura que las nuevas funcionalidades de registro y autenticación funcionan como se espera.
Prevención de Regresiones: Detecta rápidamente cualquier cambio que pueda romper la funcionalidad existente.
Recomendaciones:

Cobertura de Pruebas Ampliada: Continúa añadiendo pruebas para cubrir nuevos endpoints y casos de uso.
Pruebas de Integración y de Carga: Incorpora pruebas que evalúen cómo interactúan diferentes componentes de la API y cómo se comporta bajo diferentes cargas.
7. Recursos Adicionales
Flask-SQLAlchemy Documentation: Flask-SQLAlchemy Docs
Flask-Migrate Documentation: Flask-Migrate Docs
Flask-Bcrypt Documentation: Flask-Bcrypt Docs
Flask-JWT-Extended Documentation: Flask-JWT-Extended Docs
Flask-Limiter Documentation: Flask-Limiter Docs
SQLAlchemy Documentation: SQLAlchemy Docs
Flask Testing Documentation: Flask Testing Docs
PyTest Documentation: PyTest Docs
Redis Documentation: Redis Docs
Celery Documentation: Celery Docs
Prometheus Documentation: Prometheus Docs
Grafana Documentation: Grafana Docs
Loguru Documentation: Loguru Docs
TensorFlow Lite Model Optimization: TensorFlow Lite Optimization
Conclusión
En el Día 24, has avanzado significativamente al implementar el registro y gestión de usuarios con una base de datos, fortaleciendo la seguridad de tu API mediante hashing de contraseñas y autenticación JWT. Estas mejoras permiten una administración más robusta y escalable de los usuarios que acceden a tu API, garantizando que solo usuarios autorizados puedan realizar predicciones.

Pasos Clave Realizados:

Configuración de la Base de Datos:
Integraste SQLAlchemy, Flask-Migrate y Flask-Bcrypt para gestionar usuarios de manera persistente y segura.
Implementación del Registro de Usuarios:
Creaste un endpoint /register que permite a los nuevos usuarios registrarse proporcionando sus credenciales.
Almacenaste las contraseñas de manera segura utilizando hashing con Bcrypt.
Actualización del Endpoint de Login:
Modificaste el endpoint /login para autenticar a los usuarios verificando sus credenciales contra la base de datos y generar tokens JWT.
Actualización de las Pruebas Automatizadas:
Añadiste pruebas para verificar que el registro de usuarios y la autenticación JWT funcionan correctamente.
Recomendaciones para Continuar:

Implementar Roles y Permisos: Añade niveles de acceso para diferentes tipos de usuarios (e.g., admin, user).
Mejorar la Seguridad de JWT: Considera implementar refresh tokens y gestionar la expiración de los tokens de manera adecuada.
Optimizar el Frontend: Asegura que el manejo de tokens en el frontend sea seguro, utilizando cookies HTTP-only en lugar de almacenamiento local si es posible.
Ampliar las Funcionalidades de la API: Añade más endpoints o funcionalidades que puedan beneficiarse de la gestión de usuarios y autenticación robusta.
Implementar Confirmación de Email: Añade un proceso para verificar las direcciones de correo electrónico de los usuarios durante el registro.
¡Has realizado un excelente progreso en tu proyecto de Transfer Learning! Continúa aplicando estas prácticas de seguridad y gestión de usuarios para mantener tu API robusta, segura y eficiente.

Si tienes alguna otra pregunta o necesitas más asistencia en los próximos pasos, no dudes en contactarme. ¡Estoy aquí para ayudarte a alcanzar el éxito en tu proyecto de Transfer Learning!